# Correspondence Learning

Exercise of sound-image correspondence learning.
It is introduced in the paper:

> D. Harwath, A. Torralba, and J. Glass, "Unsupervised learning of spoken language with visual context," in *Proc. NIPS*, 2016.

## Table of contents
1. Remove silence from segmented audio
1. Extract spectrogram from segmented audio
1. Add S/N=30dB white noise to segmented audio
1. Dataset
1. Loss function
1. Model
1. Train model
1. Extract features using pretrained model

In [ ]:
import datetime
import functools
from glob import glob
from IPython.display import Audio
import os
from pathlib import Path
import pickle
import random
import sys
import time

import librosa
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pydub import AudioSegment
import resampy
import soundfile as sf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.models as models
import torchvision.transforms.functional as F
from tqdm import tqdm

os.chdir("..")

## Data path

In [ ]:
class Args:
    def __init__(self):
        # Remove silence
        self.load_dir = "exp1/seg/segmented_wavs"
        self.save_dir = "exp1/unsup_rl/test/trimed_wavs"
        # Extract spectrogram
        self.audio_list = "exp1/unsup_rl/test/segmented_wavs.txt"
        self.save_list = "exp1/unsup_rl/test/mfccs.txt"
        # Add noise
        self.noisy_dir = "exp1/unsup_rl/test/noisy_trimed_wavs"


args = Args()

## Remove silence from segmented audio

As a preprocessing, we remove silence from the audio segments and add noise to them.
Remove silence from segmented audio because a combined wave file contains 1-3 seconds of random intervals.

<img src="../fig/data.png" width=600>

In [ ]:
random.seed(202006241517)

def get_sampling_rate(audio_path: str) -> int:
    import wave
    with wave.open(audio_path, "rb") as f:
        return f.getframerate()

def rm_voiceless(audio_path: str, save_path: str):
    sr = get_sampling_rate(audio_path)
    sound, _ = librosa.core.load(audio_path, sr = sr)
    sound, _ = librosa.effects.trim(sound)
    # sf.write(save_path, sound ,sr)

audio_dir = args.load_dir
save_dir = args.save_dir

paths = [i for i in Path(audio_dir).glob("*.wav")]
for p in tqdm(paths, desc="rm_voiceless"):
    savep = Path(save_dir) / p.name
    rm_voiceless(str(p), str(savep))

## Extract spectrogram from segmented audio for A_Dataset

A_Dataset will be defined later.

In [ ]:
def enc(a, sr):
    win_len_sec = 0.025
    hop_sec = 0.010
    ret = librosa.stft(a, n_fft=int(win_len_sec * sr),
                        hop_length=int(hop_sec * sr))
    return ret

def extract_and_save_from_mp3file(io):
    in_path, out_path = io
    if Path(out_path).is_file(): return
    try:
        feature = mp3file_to_examples(in_path)
        # with open(out_path, "wb") as f:
        #     pickle.dump(feature, f)
    except Exception as e:
        print("Failed: {}".format(in_path), file=sys.stderr)

def mp3file_to_examples(mp3_file):
    a, sr = librosa.load(mp3_file)
    resr = 16000
    a = resampy.resample(a, sr, resr)
    feature = enc(a, resr)
    return feature

if __name__ == "__main__":
    audio_list = []
    with open(args.audio_list, "r") as f:
        for p in f:
            audio_list.append(p.strip())

    save_list = []
    with open(args.save_list, "r") as f:
        for p in f:
            save_list.append(p.strip())
        
    plist = list(zip(audio_list, save_list))

    from multiprocessing import Pool
    import multiprocessing as multi
    import sys

    pool = Pool(multi.cpu_count())
    for i, _ in enumerate(pool.imap_unordered(extract_and_save_from_mp3file, plist)):
        if i % 1000 == 0: print(i, flush=True)
    pool.close()

## Add S/N=30dB white noise to segmented audio

This is the noise in the agent-human interaction environment.

In [ ]:
def add_voiceless_and_noise(audio_path: str, save_path: str, noise_db: int = 30, voiceless_len_ms: int = 1000):
    from pydub.generators import WhiteNoise
    sound = AudioSegment.from_wav(audio_path)
    noise_dbfs = sound.dBFS-noise_db
    
    def make_sil(duration: int) -> AudioSegment:
        return AudioSegment.silent(duration=duration)
    
    def make_noise(duration: int, dbfs: float) -> AudioSegment:
        return WhiteNoise().to_audio_segment(duration=duration, volume=dbfs)

    noise = make_noise(len(sound), noise_dbfs)
    sound = sound.overlay(noise)
    
    first_noise = make_noise(voiceless_len_ms, noise_dbfs)
    last_noise  = make_noise(voiceless_len_ms, noise_dbfs)
    sound = first_noise + sound + last_noise
    sound.export(save_path, format="wav")

if __name__ == "__main__":
    random.seed(202006241517)

    audio_dir = args.save_dir
    save_dir = args.noisy_dir
    paths = [i for i in Path(audio_dir).glob("*.wav")]
    for p in tqdm(paths, desc="Add white noise"):
        savep = Path(save_dir) / p.name
        # add_voiceless_and_noise(p, savep)

## Example of segmented audio
1. with silence
1. without silence
1. with S/N=30dB white noise

In [ ]:
audio_with_silence, rate = librosa.load("exp1/seg/segmented_wavs/1_7438.wav")
Audio(audio_with_silence, rate=rate)

In [ ]:
audio_without_silence, rate = librosa.load("exp1/unsup_rl/test/trimed_wavs/1_7438.wav")
Audio(audio_without_silence, rate=rate)

In [ ]:
noisy_audio, rate = librosa.load("exp1/unsup_rl/test/noisy_trimed_wavs/1_7438.wav")
Audio(noisy_audio, rate=rate)

## Dataset

- AV_Dataset: Tuple of audio description and food image. For each food, we used 90 images for training. There are four types of audio descriptions.
    - foodname
    - A (An) foodname
    - A (An) color foodname
    - It's a (an) foodname

- V_Dataset: training images in AV_Dataset.

- A_Dataset: Spectrogram of segmented audio including meaningless sound.

<img src="../fig/corr.png" width=600>

In [ ]:
class AV_Dataset(Dataset):
    def __init__(self, pair_paths, a_tr, v_tr):
        self.pair_paths = pair_paths  # (audio, visual)
        self.a_tr = a_tr
        self.v_tr = v_tr

    def __len__(self):
        return len(self.pair_paths)

    def _a_load(self, path):
        with open(path, "rb") as f:
            feat = pickle.load(f)
        log_pad = 0.010
        feat = np.log(np.abs(feat) + log_pad).T
        if self.a_tr:
            feat = self.a_tr(feat)
        return feat

    def _v_load(self, path):
        img = Image.open(path)
        img = img.resize((224, 224))
        if self.v_tr:
            img = self.v_tr(img)
        return img

    @functools.lru_cache(maxsize=None)  # Cache loaded structures
    def __getitem__(self, index):
        a = self._a_load(self.pair_paths[index][0])
        v = self._v_load(self.pair_paths[index][1])
        return a, v

class A_Dataset(Dataset):
    def __init__(self, paths, a_tr, v_tr):
        self.paths = paths  # (audio, visual)
        self.a_tr = a_tr
        self.v_tr = v_tr

    def __len__(self):
        return len(self.paths)

    def _a_load(self, path):
        with open(path, "rb") as f:
            feat = pickle.load(f)
        log_pad = 0.010
        feat = np.log(np.abs(feat) + log_pad).T
        if self.a_tr:
            feat = self.a_tr(feat)
        return feat

    def _v_load(self, path):
        img = Image.open(path)
        img = img.resize((224, 224))
        if self.v_tr:
            img = self.v_tr(img)
        return img

    @functools.lru_cache(maxsize=None)  # Cache loaded structures
    def __getitem__(self, index):
        a = self._a_load(self.paths[index])
        return a

class V_Dataset(Dataset):
    def __init__(self, paths, a_tr, v_tr):
        self.paths = paths  # (audio, visual)
        self.a_tr = a_tr
        self.v_tr = v_tr

    def __len__(self):
        return len(self.paths)

    def _a_load(self, path):
        with open(path, "rb") as f:
            feat = pickle.load(f)
        log_pad = 0.010
        feat = np.log(np.abs(feat) + log_pad).T
        if self.a_tr:
            feat = self.a_tr(feat)
        return feat

    def _v_load(self, path):
        img = Image.open(path)
        img = img.resize((224, 224))
        if self.v_tr:
            img = self.v_tr(img)
        return img

    @functools.lru_cache(maxsize=None)  # Cache loaded structures
    def __getitem__(self, index):
        v = self._v_load(self.paths[index])
        return v

## Loss function

The sim_loss is based on the triplet loss.
The imposter of the audio description is randomly selected from the rest of the mini-batch.

In [ ]:
def random_index(ban_i, ceil_i):
    max_i = ceil_i - 1
    assert 0 <= ban_i <= max_i
    i = random.randint(0, max_i - 1)
    if i == ban_i:
        i = max_i
    return i


def sim_loss_batchsum(feat_vs, feat_as, rho = 4.0, eps = 1e-5):
    ret = 0

    for i in range(len(feat_vs)):
        j = random_index(i, len(feat_vs))
        ret += sim_loss(feat_vs[i], feat_as[i], feat_vs[j], feat_as[j], rho, eps)
    return ret


def sim_loss(feat_v, feat_a, feat_v_imp,feat_a_imp, rho = 4.0, eps=1e-5):
    assert len(feat_v) == len(feat_a)

    score_corr = ((feat_v - feat_a)**2).sum()**(1/2)
    score_impa = ((feat_v - feat_a_imp)**2).sum()**(1/2)
    if score_corr.is_cuda:
        zeros = torch.cuda.FloatTensor((1,)).fill_(0)
    else:
        zeros = torch.zeros(*score_corr.shape)
    ret = 0.5*score_corr**2 + 0.5*torch.max(zeros, rho - score_impa)**2
    ret = torch.mean(ret)
    return ret

## Model

The sound front-end and image front-end are randomly initialized ResNet50.
> K. He, X. Zhang, S. Ren, and J. Sun, "Deep Residual Learning for Image Recognition," in *Proc. CVPR*, 2016.

Using the model, audio and images are embedded in the same dimensional space.

<img src="../fig/corr.png" width=600>

In [ ]:
class SimpleImageCorrNet(nn.Module):
    def __init__(self, num_class = 50, ext_v = True, ext_a = True, pre_trained=False):
        super(SimpleImageCorrNet, self).__init__()
        self.visual_net = models.resnet50(num_classes=num_class, pretrained=pre_trained)
        self.audio_net = models.resnet50(num_classes=num_class, pretrained=pre_trained)
        self.audio_net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.ext_v = ext_v
        self.ext_a = ext_a

    def _visual_extract(self, v):
        return self.visual_net(v)

    def _sound_extract(self, a):
        return self.audio_net(a[:,None,:,:])

    def forward(self, visual_feat, audio_feat):
        if self.ext_v:      visual_feat = self._visual_extract(visual_feat)
        if self.ext_a:      audio_feat = self._sound_extract(audio_feat)
        return (visual_feat, audio_feat)

In [ ]:
class Opts:
    def __init__(self, modality, pkllist, outpath):
        # Correspondence learning
        self.dataset_path = "exp1/unsup_rl/test/audio_image_pair.txt"
        self.workdir = "exp1/unsup_rl/test"
        # Conf
        self.del_last = 1
        self.ext_fusion = 0
        self.rho = 2.0
        self.epoch = 2
        self.batch_size = 16
        # Extract nn feat
        self.modality = modality
        self.modelparam = "exp1/unsup_rl/test/unsup_backend.pth.tar"
        self.pkllist = pkllist
        self.outpath = outpath


opts = Opts(
    "audio",
    "exp1/unsup_rl/test/seg_audios.txt",
    "exp1/unsup_rl/test/data/new720db20k2_fea_sim_8type_clean_limited_data.npy",
)

## Train model

In [ ]:
LOG_NAME=opts.workdir + "/log/" + datetime.datetime.fromtimestamp(time.time()).strftime("%Y%m%d_%H_%M_%S") + ".log"
Path(LOG_NAME).parent.mkdir(parents=True, exist_ok=True)

def logger(s):
    print(s, flush=True)
    with open(LOG_NAME, "a") as f:
        f.write(s)
        f.write("\n")


class KeepUnderThr(object):
    def __init__(self, times, thr):
        self.times = times
        self.thr = thr
        self.count = 0

    def __call__(self, loss):
        if loss < self.thr:
            self.count += 1
        else:
            self.count = 0
        return self.count >= self.times


def get_remainder(datasize, b_batch_size, batch_size):
    assert b_batch_size % batch_size == 0
    remainder = datasize % b_batch_size
    rem_start_iter = (datasize // b_batch_size) * (b_batch_size // batch_size)
    return remainder, rem_start_iter


def train_video_corr_net(model, device, loader, optimizer, epoch, start_time, break_fn):
    model.train()
    sum_loss = 0
    batch_num = 0
    iter_num = 0
    NUM_B_BATCH = opts.batch_size*10
    rem, rem_start_iter = get_remainder(len(loader.dataset), NUM_B_BATCH, loader.batch_size)
    for batch_idx, (snd, img) in enumerate(loader):
        img, snd = img.to(device), snd.to(device)
        loss = 0
        if batch_num == 0:
            iter_mean_loss = 0
            optimizer.zero_grad()
        vcs, acs = model(img, snd)
        loss = sim_loss_batchsum(vcs,acs,rho=opts.rho)
        if batch_idx < rem_start_iter:
            loss /= NUM_B_BATCH
        else:
            loss /= rem
        iter_mean_loss += loss.item()
        batch_num += img.shape[0]
        loss.backward()
        if batch_idx+1 != len(loader) and batch_num != NUM_B_BATCH: continue
        iter_num += 1
        sum_loss += iter_mean_loss
        optimizer.step()

        logger(f"Train Epoch: {epoch} [{(batch_idx+1)*len(img)}/{len(loader.dataset)} ({int(100.*(batch_idx+1)/len(loader)):3}%)]\tLoss: {iter_mean_loss:.6f}\tTime:{time.time() - start_time}")
        batch_num = 0
    loss_mean = sum_loss/iter_num
    logger("Train Epoch: {} Loss: {:.6f}\tTime:{}".format(
            epoch, loss_mean,time.time() - start_time))
    return loss_mean


def repeat_last(batch):
    import numpy as np
    a, v = [], []
    a_len = -1
    for b in batch:
        a.append(b[0])
        v.append(b[1])
        a_len = max(a_len, len(b[0]))
    a_pad = []
    for ae in a:
        pad = ae[-1:].repeat(a_len-len(ae), axis=0)
        ae = np.concatenate((ae, pad), axis=0)
        a_pad.append(torch.from_numpy(ae))
    a_pad = torch.stack(a_pad, dim=0).float()
    v = torch.stack(v, dim=0).float()
    return a_pad,v


def train_data2(shuffle=True, dataset_path="data/plist_train_noise.txt"):
    from torch.utils.data import DataLoader
    paths = []
    with open(dataset_path, "r") as f:
        for t in f:
            paths.append(tuple(t.strip().split()))
    v_tr = transforms.ToTensor()
    v_tr_train = transforms.ToTensor()
    loader = DataLoader(AV_Dataset(paths, None, v_tr_train), batch_size=opts.batch_size, shuffle=shuffle, num_workers=7, pin_memory=True, collate_fn=repeat_last)
    return loader


def saves(model, optimizer, path):
    stt = {
            "model_state_dict": model.state_dict(),
            "optim_state_dict": optimizer.state_dict()
            }
    torch.save(stt, path)

If `opts.epoch` is set to a sufficiently large value, the training will automatically terminate after about 600-800 epochs.

In [ ]:
if __name__ == "__main__":
    LOAD_EPOCH=0
    END_EPOCH=opts.epoch
    loader = train_data2(dataset_path=opts.dataset_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ext_fusion = True if opts.ext_fusion == 1 else False
    del_last = True if opts.del_last == 1 else False
    model = SimpleImageCorrNet(num_class = 50).to(device)
    lr = 1e-4
    optimizer = optim.Adam(model.parameters(), lr=lr)
    start_time = time.time()
    break_fn = KeepUnderThr(times=10, thr=1e-5)
    loss_list = []
    patience = 0
    loss_min = 100000
    for epoch in range(LOAD_EPOCH+1, END_EPOCH+1):
        loss_h = train_video_corr_net(model, device, loader, optimizer, epoch, start_time, break_fn)
        len_loader = len(loader)
        loss_list.append(loss_h)
        # np.save(opts.workdir + "/unsup_loss.npy", loss_list)
        loss_min = min(loss_list)

        if loss_h > loss_min:
            patience += 1
        else:
            patience = 0
            # saves(model, optimizer, f"{opts.workdir}/unsup_backend.pth.tar")
            # print("save")
        if patience > 20*7200//opts.batch_size//len_loader and lr > 1e-7:
            lr = lr/10
            patience = 0
            optimizer = optim.Adam(model.parameters(), lr=lr)
            print("Decrease lr")
        if patience > 100*7200//opts.batch_size//len_loader:
            print("Early stop", loss_min)
            break
        print(loss_min)

## Extract features using pretrained model

Using the model pretrained by correspondence learning, we extract features from V_Dataset and A_Dataset.
The K-Means clustering and selection of audio segments by distance matrix are done in [exercise3.ipynb](exercise3.ipynb).

<img src="../fig/corr.png" width=600>

In [ ]:
def load_model(parampath, modality, device):
    model = SimpleImageCorrNet(num_class = 50).to(device)
    checkpoint = torch.load(parampath)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    if modality == "audio":
        return model._sound_extract
    elif modality == "image":
        return model._visual_extract
    else:
        raise ValueError()

def load_data(dataset_path, modality):
    paths = []
    with open(dataset_path, "r") as f:
        for t in f:
            paths.append(t.strip())
    v_tr = transforms.ToTensor()
    if modality == "audio":
        DatasetClass = A_Dataset
    elif modality == "image":
        DatasetClass = V_Dataset
    else:
        raise ValueError()
    seg_loader = DataLoader(DatasetClass(paths, None, v_tr), batch_size=1, shuffle=False, num_workers=7, pin_memory=True)
    return seg_loader

def extract(model, data, device):
    seg_fea = []
    with torch.no_grad():
        for seg in tqdm(data):
            seg = seg.to(device)
            fea = model(seg)
            seg_fea.append(fea.cpu())
    seg_fea_tensor = torch.stack(seg_fea, dim=0)
    return seg_fea_tensor


def extract_nnfeat(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = load_model(args.modelparam, args.modality, device)
    seg_loader = load_data(args.pkllist, args.modality)
    seg_fea_tensor = extract(model, seg_loader, device)

    # np.save(args.outpath, seg_fea_tensor.numpy())

In [ ]:
extract_nnfeat(opts)

opts = Opts(
    "image",
    "exp1/unsup_rl/test/train_imgs.txt",
    "exp1/unsup_rl/test/data/train_img_fea_sim_8type_clean_limited_data.npy",
)
extract_nnfeat(opts)

## Test pretrained model

Plot the distance matrix between audio features and image features.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parampath = "exp1/unsup_rl/test/unsup_backend.pth.tar"
model = SimpleImageCorrNet(num_class = 50).to(device)
checkpoint = torch.load(parampath)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

loader = iter(train_data2(dataset_path=opts.dataset_path))

## Function for plot

In [ ]:
def imshow(image: torch.Tensor):
    plt.figure(figsize=(2*len(image),2))
    for i in range(len(image)):
        plt.subplot(1, len(image), i+1)
        image_i = image[i].detach().squeeze()
        image_i = F.to_pil_image(image_i)
        plt.imshow(np.asarray(image_i))
        plt.axis("off")

def specshow(spec: torch.Tensor):
    plt.figure(figsize=(2*len(spec),2))
    for i in range(len(spec)):
        plt.subplot(1, len(spec), i+1)
        spec_i = spec[i].detach().squeeze()
        plt.imshow(np.asarray(spec_i))
        plt.axis("off")

## Plot distance matrix

The (i, j) component of the distance matrix is the Euclidean distance of the feature between the i-th image and the j-th audio.
As can be seen from the plot, the diagonal component of the distance matrix is small.

In [ ]:
sound, image = next(loader)

imshow(image)
specshow(sound)

image, sound = image.to(device), sound.to(device)
image, sound = model(image, sound)

plt.figure()
plt.imshow(torch.cdist(image, sound).cpu().detach().numpy())
plt.title("Distance matrix")
plt.xlabel("Index of sound")
plt.ylabel("Index of image")
plt.colorbar()